In [ ]:
import os
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import pytorch_lightning as pl
import torch
from model.model_interface import LLM
import torch.utils.data as tud
from torch.utils.data import DataLoader
from lightning.pytorch.loggers import WandbLogger
import torch.nn.functional as F
from tqdm.notebook import tqdm
from utils.viz_tool import *


torch.set_float32_matmul_precision('medium')
os.environ['TOKENIZERS_PARALLELISM'] = 'true'

LLM Config

In [ ]:
llama_7b_local_dir = "/nvme/guoyiqiu/coding/huggingface/my_models/llama-7b"
llm_config = {
    "model_name": llama_7b_local_dir,
    "half": True,
}
mt = LLM(**llm_config)

In [ ]:
mt.model.model.layers[0].mlp.up_proj.weight.dtype

Hook Config

In [ ]:
hook_config = {
    "retain_output": True,
    "retain_input": False,
    "edit_output": None,
    "clone": False,
    "detach": False,
    "device": "cpu"
}

n_layer = mt.model.config.num_hidden_layers
# gpt2 config
# blocks = mt.model.transformer.h
# llama config
blocks = mt.model.model.layers


mt.clear_hook()
for i in range(n_layer):
    mt.add_hook(module=blocks[i],name=f"block_{i}", **hook_config)

Dataset Config

In [ ]:
bsz = 1

from dataset.knowns import Knowns

dst = Knowns("/nvme/guoyiqiu/coding/gpt_re/data", mt.tokenizer)
dl = DataLoader(dst, batch_size=bsz, collate_fn=dst.collate_fn)

Trainer config

In [ ]:
trainer_config = {
    "precision" : "16-mixed",
    "accelerator" : "auto",
    "devices" : [5],
}
trainer = pl.Trainer(**trainer_config)

In [ ]:
res = trainer.predict(mt, dl)

Analyse Mean Residual Norm

In [ ]:
blocks_mean_output = [[] for i in range(n_layer)]
for idx, (input_ids, attention_mask, labels) in enumerate(dl):
    seq_len = attention_mask.sum(dim=1).unsqueeze(-1).repeat(1,mt.model.config.hidden_size)
    attention_mask = attention_mask.unsqueeze(-1).repeat(1,1,mt.model.config.hidden_size)
    for i in range(n_layer):
        output_i_idx = mt.hooks[f"block_{i}"].outputs[idx][0]
        output_i_idx = output_i_idx * attention_mask.float()
        output_i_idx = output_i_idx.sum(dim=1) / seq_len # [bsz, hidden_size] # compute mean
        # output_i_idx = output_i_idx[:,-1,:] # [bsz, hidden_size] # use last
        blocks_mean_output[i].append(output_i_idx)
blocks_mean_output = [torch.vstack(b).mean(0) for b in blocks_mean_output]
plotly_bar("Avg norm of layer output", [torch.norm(b).item() for b in blocks_mean_output])

Unembedding

In [ ]:
idx = 125

bi = idx // bsz
i = idx - (idx // bsz) * bsz

input_ids, attention_mask, labels = list(dl)[bi][0][i], list(dl)[bi][1][i], list(dl)[bi][2][i]
outputs = [h.outputs[bi][0][i] for h in mt.hooks.values()]
logits = res[bi]['logits'][i][-2] # last token is <\s> so -2

input_tokens = mt.tokenizer.decode(input_ids)
prob, next_ids = torch.topk(F.softmax(logits.float(),dim=-1), 5)
next_token = {mt.tokenizer.decode(t) : "{:7f}".format(p.item()) for p, t in zip(prob, next_ids)}

print(f"prompt: {input_tokens}")
print(f"next token: {next_token}")

# mt.generate(input_tokens, max_new_tokens=5)

In [ ]:
import plotly.graph_objs as go
import numpy as np

data = np.array([[1, 2, 3],
                 [4, 5, 6],
                 [7, 8, 9]])

# 添加辅助信息字典
info_dict = {'key1': 'value1', 'key2': 'value2', 'key3': 'value3'}

# 创建customdata数组
customdata = []
for row in data:
    customdata_row = []
    for val in row:
        customdata_row.append(info_dict)
    customdata.append(customdata_row)

trace = go.Heatmap(z=data, customdata=customdata, hovertemplate='x=%{x}<br>y=%{y}<br>customdata=%{customdata}')

layout = go.Layout(title='Heatmap Example',
                   xaxis=dict(title='X Axis'),
                   yaxis=dict(title='Y Axis'))

fig = go.Figure(data=[trace], layout=layout)

fig.show()
